In [1]:
import json
import pandas

ModuleNotFoundError: No module named 'scipy'

# Get the character_stats, weapon stats and artifact stats that you want

In [ ]:
character = "characters/ellen.json"
weapon = "weapons/deep_sea_visitor.json"
disk = "disks_setup/standard.json"
excel_save_name = "results/ellen_sig.xlsx"

def get_stats(character, weapon, disk):
    with open(character) as f:
        character_dict = json.load(f)
        character_stats = character_dict["character_stats"]
        motion_values = character_dict["motion_values"]
    with open(weapon) as f:
        weapon_stats = json.load(f)
    with open(disk) as f:
        other_stats = json.load(f)
    return character_stats, motion_values, weapon_stats, other_stats

character_stats, motion_values, weapon_stats, other_stats = get_stats(character, weapon, disk)

anomaly_data = {
    "Fire": 0.5 * 20,
    "Electric": 1.25 * 10,
    "Ether": 0.625 * 20,
    "Ice": 5,
    "Physical": 0.5 * 20
}

target_stats = {
    "Enemy": "Tyrfing",
    "Level": 70,
    "Scaling_ID": 1000,
    "Base_DEF": 36,
    "DEF": 571.7,
    "Stun_Scaling_ID": 1003,
    "Base_Stun": 600,
    "Stun": 1002,
    "Fire_RES": 0,
    "Ice_RES": -0.2,
    "Electric_RES": 0,
    "Ether_RES": -0.2,
    "Physical_RES": 0,
    "Daze_Resist": 0,
    "Daze_Taken": 0,
    "Stun_Multiplier": 1,
}

In [ ]:
def get_total_stats(character_stats, weapon_stats, other_stats):
    # Complete initialization of total_stats
    total_stats = {
        "Level": 60,
        "Level_Coefficient": 281,
        "DEF_Reduction": 0,
        "Stun_Bonus": 0,
    }
    total_stats["Total_HP"] = ((character_stats["Base_HP"] + weapon_stats["Base_HP"] ) \
                            * (1 + weapon_stats["HP_%"] + other_stats["HP_%"]) + other_stats["Flat_HP"]) \
                                * (1 + weapon_stats["Final_HP_%"] + other_stats["Final_HP_%"]) + other_stats["Final_Flat_HP"]

    total_stats["Total_DEF"] = ((character_stats["Base_DEF"] + weapon_stats["Base_DEF"]) \
                                * (1 + weapon_stats["DEF_%"] + other_stats["DEF_%"]) + other_stats["Flat_DEF"]) \
                                    * (1 + weapon_stats["Final_DEF_%"] + other_stats["Final_DEF_%"]) + other_stats["Final_Flat_DEF"]

    total_stats["Total_ATK"] = ((character_stats["Base_ATK"] + weapon_stats["Base_ATK"]) \
                                * (1 + weapon_stats["ATK_%"] + other_stats["ATK_%"]) + other_stats["Flat_ATK"]) \
                                    * (1 + weapon_stats["Final_ATK_%"] + other_stats["Final_ATK_%"]) + other_stats["Final_Flat_ATK"]

    total_stats["CRIT_Rate"] = character_stats["CRIT_Rate"] + weapon_stats["CRIT_Rate"] + other_stats["CRIT_Rate"]
    total_stats["CRIT_DMG"] = character_stats["CRIT_DMG"] + weapon_stats["CRIT_DMG"] + other_stats["CRIT_DMG"]
    total_stats["PEN_Ratio"] = character_stats["PEN_Ratio"] + weapon_stats["PEN_Ratio"] + other_stats["PEN_Ratio"]
    total_stats["Impact"] = character_stats["Impact"] + weapon_stats["Impact"]
    total_stats["Anomaly_Mastery"] = character_stats["Anomaly_Mastery"] + weapon_stats["Anomaly_Mastery"]
    total_stats["Anomaly_Proficiency"] = character_stats["Anomaly_Proficiency"] + other_stats["Anomaly Proficiency"] \
        + weapon_stats["Anomaly Proficiency"]
    total_stats["Energy_Regen"] = character_stats["Energy_Regen"] + weapon_stats["Energy_Regen"]


    total_stats["Buff Level Multiplier"] = 1 + 0.0169 * (total_stats["Level"] - 1)

    total_stats["DMG%_Multiplier"] = 1 + weapon_stats["Damage%_Multiplier"] + other_stats["Elemental_Damage"]
    total_stats["DMG_Taken_Multiplier"] = 1
    # Complete initialization of target_stats
    target_stats["Effective_DEF"] = target_stats["DEF"] * (1 - total_stats["PEN_Ratio"]) - other_stats["Flat_PEN"]
    target_stats["DEF_Multiplier"] = total_stats["Level_Coefficient"] / (total_stats["Level_Coefficient"] + target_stats["Effective_DEF"])
    return total_stats

total_stats = get_total_stats(character_stats, weapon_stats, other_stats)
print(total_stats)

{'Level': 60, 'Level_Coefficient': 281, 'DEF_Reduction': 0, 'Stun_Bonus': 0, 'Total_HP': 9872.3, 'Total_DEF': 789.4, 'Total_ATK': 2908.3997, 'CRIT_Rate': 0.898, 'CRIT_DMG': 1.652, 'PEN_Ratio': 0, 'Impact': 93, 'Anomaly_Mastery': 94, 'Anomaly_Proficiency': 93, 'Energy_Regen': 1.2, 'Buff Level Multiplier': 1.9970999999999999, 'DMG%_Multiplier': 1.55, 'DMG_Taken_Multiplier': 1}


# Real calculations begin

In [ ]:
# Iterate through the motion values and calculate the damage
def damage_calculator(character_stats, motion_values, weapon_stats, other_stats, target_stats):
    # First get the total_stats
    total_stats = get_total_stats(character_stats, weapon_stats, other_stats)
    # for each attack in motion_values
    for key, value in motion_values.items():
        motion_values[key]["Base_DMG"] = value["DMG"] / 100 * total_stats["Total_ATK"]
        motion_values[key]["RES_Multiplier"] = 1 + target_stats[value["Element"] + "_RES"]
        motion_values[key]["Daze_Buildup"] = value["Daze"] / 100 * total_stats["Impact"] * (1 - target_stats["Daze_Resist"]) * (1 + total_stats["Stun_Bonus"])
        
        motion_values[key]["Average_Outgoing_DMG"] = motion_values[key]["Base_DMG"] \
            * total_stats["DMG%_Multiplier"] \
            * total_stats["DMG_Taken_Multiplier"] \
            * target_stats["DEF_Multiplier"] \
            * motion_values[key]["RES_Multiplier"] \
            * target_stats["Stun_Multiplier"] \
            * (1 + total_stats["CRIT_Rate"] * total_stats["CRIT_DMG"])

    # Add anomaly damage to the motion_values
    motion_values["Anomaly_DMG"] = (anomaly_data[character_stats["Element"]] * total_stats["Total_ATK"]) \
        * total_stats["DMG%_Multiplier"] \
        * target_stats["DEF_Multiplier"] \
        * (1 + target_stats[character_stats["Element"] + "_RES"]) \
        * total_stats["DMG_Taken_Multiplier"] \
        * target_stats["Stun_Multiplier"] \
        * total_stats["Anomaly_Proficiency"]/100 \
        * total_stats["Buff Level Multiplier"]
    return motion_values

motion_values = damage_calculator(character_stats, motion_values, weapon_stats, other_stats, target_stats)
print(motion_values)

{'hit_1': {'DMG': 89.3, 'Daze': 35.2, 'Element': 'Physical', 'Base_DMG': 2597.2009321, 'RES_Multiplier': 1, 'Daze_Buildup': 32.736000000000004, 'Average_Outgoing_DMG': 3294.6612651674873}, 'hit_2': {'DMG': 202.0, 'Daze': 122.8, 'Element': 'Physical', 'Base_DMG': 5874.967394, 'RES_Multiplier': 1, 'Daze_Buildup': 114.204, 'Average_Outgoing_DMG': 7452.649222439333}, 'hit_3': {'DMG': 541.2, 'Daze': 339.4, 'Element': 'Physical', 'Base_DMG': 15740.259176400003, 'RES_Multiplier': 1, 'Daze_Buildup': 315.642, 'Average_Outgoing_DMG': 19967.196827644395}, 'e_hit_1': {'DMG': 89.3, 'Daze': 35.2, 'Element': 'Ice', 'Base_DMG': 2597.2009321, 'RES_Multiplier': 0.8, 'Daze_Buildup': 32.736000000000004, 'Average_Outgoing_DMG': 2635.72901213399}, 'e_hit_2': {'DMG': 202.0, 'Daze': 122.8, 'Element': 'Ice', 'Base_DMG': 5874.967394, 'RES_Multiplier': 0.8, 'Daze_Buildup': 114.204, 'Average_Outgoing_DMG': 5962.119377951467}, 'e_hit_3': {'DMG': 541.2, 'Daze': 339.4, 'Element': 'Ice', 'Base_DMG': 15740.25917640000

In [ ]:
# Show in pandas dataframe
import pandas as pd
df = pd.DataFrame(motion_values)
print(df)
# Create an excel sheet for this calculation
df.to_excel(excel_save_name)

                            hit_1        hit_2         hit_3      e_hit_1  \
DMG                          89.3        202.0         541.2         89.3   
Daze                         35.2        122.8         339.4         35.2   
Element                  Physical     Physical      Physical          Ice   
Base_DMG              2597.200932  5874.967394  15740.259176  2597.200932   
RES_Multiplier                  1            1             1          0.8   
Daze_Buildup               32.736      114.204       315.642       32.736   
Average_Outgoing_DMG  3294.661265  7452.649222  19967.196828  2635.729012   
Energy_Cost                   NaN          NaN           NaN          NaN   

                          e_hit_2       e_hit_3 charged_dash_attack  \
DMG                         202.0         541.2               287.8   
Daze                        122.8         339.4               172.1   
Element                       Ice           Ice                 Ice   
Base_DMG              

PermissionError: [Errno 13] Permission denied: 'results/ellen_sig.xlsx'

In [ ]:
# Define a class for the model with trainable main stat and sub stats
class DamageModel(torch.nn.Module):
  def __init__(self):
    super(DamageModel, self).__init__()
    self.main_stat = torch.nn.Parameter(torch.tensor(main_stat_value))
    self.sub_stats = torch.nn.Parameter(sub_stats)

  def forward(self):
    # Access character stats and perform damage calculation
    damage = damage_calculator(character_stats_tensor, self.main_stat, self.sub_stats)
    return damage

# Create the model instance
model = DamageModel()

# Define loss function with L1 regularization (adjust alpha as needed)
alpha = 0.1
def loss_function(damage):
  return -damage + alpha * torch.sum(torch.abs(model.sub_stats))

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1000):
  # Forward pass
  damage = model()

  # Calculate loss
  loss = loss_function(damage)

  # Backward pass and optimize
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()